# Computer vision en python

El objetivo de este notebook es mostrar por un lado la capacidad de ciertas librerías de python en torno al _data augmentation_ y por otro lado aplicar esta técnica en un caso real de reconocimiento de objetos.

Para ello, vamos a generar de forma aleatoria a partir de 5 imágenes básicas un conjunto de 250 imágenes, etiquetarlas y entrenar un modelo para luego realizar detección sobre dicho modelo. Dichas imágenes son las que se muestran a continuación.

<img src="originales/taxi_01.jpg" style="width: 45vw; min-width: 330px;" />
<img src="originales/taxi_02.jpg" style="width: 45vw; min-width: 330px;" />
<img src="originales/taxi_03.jpg" style="width: 45vw; min-width: 330px;" />
<img src="originales/taxi_04.jpg" style="width: 45vw; min-width: 330px;" />
<img src="originales/taxi_05.jpg" style="width: 45vw; min-width: 330px;" />

## Primer paso: Data augmentation

Esta técnica se usa comúnmente en técnicas de visión computacional cuando los datos de los que disponemos no son suficientes para nuestro estudio. A _grosso modo_, la técnica consiste en alterar una imagen de forma que para el entrenamiento de nuestro modelo, pueda contar como una imagen diferente y que por tanto aporta información extra al modelo.

In [6]:
# Cargamos las librerías

import os
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
import random
from PIL import Image
from numpy import *
from sympy import *

In [9]:
# Cargamos las 5 imágenes que vamos a emplear

originales = {}

for img_name in os.listdir("./originales"):
    name = img_name.split('.')[0]
    originales[name] = imageio.imread("./originales/"+img_name)

In [3]:
def random_image(img):
    """
    Esta función toma como argumento una imagen a la que de forma aleatoria se le van a aplicar con probabilidad 0.25 una de las siguientes transformaciones:
    - Rotación.
    - Rotación y reflexión.
    - Rotación y ruido.
    - Torsión.
    
    Todas ellas también con un grado aleatorio de alteración.
    """
    x = random.randint(-50, 50)
    y = random.randint(-50, 50)
    z =  random.randint(1, 80)
    if z <= 25: # rotamos la imagen
        rotate=iaa.Affine(rotate=(x, y))
        return rotate.augment_image(img)
    elif z <= 50: # la rotamos y le hacemos una reflexión sobre el eje vertical
        flip_hr=iaa.Fliplr(p=1.0)
        rotate=iaa.Affine(rotate=(x, y))
        return flip_hr.augment_image(rotate.augment_image(img))
    elif z <= 75: # la rotamos y le agregamos ruido a la imagen
        x = random.randint(-80, 80)
        y = random.randint(-80, 80)
        gaussian_noise=iaa.AdditiveGaussianNoise(-60,60)
        rotate=iaa.Affine(rotate=(x, y))
        return gaussian_noise.augment_image(rotate.augment_image(img))
    else: # torcemos la imagen
        x = random.randint(30, 60)
        y = random.randint(30, 60)
        shear = iaa.Affine(shear=(x,y))
        return shear.augment_image(img)

Generamos las 250 imágenes para el etiquetado y entrenamiento

In [ ]:
for i in originales.keys():
    for j in range(50):
        filename = "random_image_{}_{}.jpg".format(j,i)
        imageio.imwrite("train/images/"+filename, random_image(originales[i])) 

A continuación podemos ver algunos ejemplos de las imágenes generadas:

<img src="train/images/random_image_9_taxi_01.jpg" style="width: 45vw; min-width: 330px;" />
<img src="train/images/random_image_8_taxi_03.jpg" style="width: 45vw; min-width: 330px;" />
<img src="train/images/random_image_33_taxi_02.jpg" style="width: 45vw; min-width: 330px;" />


## Segundo paso: Etiquetado

Necesitamos etiquetar de forma que el modelo pueda _aprender_ de las imágenes. Para ello, vamos a emplear la herramienta [LabelImg](https://pypi.org/project/labelImg/), que de forma muy sencilla nos permite etiquetar las imágenes tal y como se muestra en la imagen a continuación

![alt_text](./resources/labelling.png)

## Tercer paso: Entrenamiento y evaluación del modelo

Dado que para elaboración de la red neuronal necesitamos de una _GPU_, nos hemos apoyado en la herramienta [Google Colab](colab.research.google.com), que nos permite configurar un notebook para usarlo con este propósito. Las librerías que hemos empleado son las siguientes:

* numpy==1.16.1
* tensorflow-gpu==1.13.1
* h5py==2.10.0
* keras==2.2.4
* imageai

En primer lugar, para conectar nuestro notebook de google colab con nuestro google drive, que es desde donde capturará los datos el modelo, necesitamos ejecutar el siguiente bloque de código:

```python
from google.colab import drive
drive.mount('/content/drive')
```

Además, puesto que vamos a realizar __transfer learning__ sobre un modelo preentrenado, al que a su vez vamos a reentrenar nosotros con nuestros datos, necesitamos descargar dicho modelo. En concreto, el modelo que vamos a emplear es YOLOv3, que puede encontrarse [aquí](https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/pretrained-yolov3.h5).

De forma que nuestro directorio de drive debe quedar con la siguiente estructura:

```
root/
├── taxi/
│   ├── train/
│   │     ├── images/
│   │     └── annotations/
│   └── validation/
│         ├── images/
│         └── annotations/
└── pretrained-yolov3.h5
```

En nuestro caso, hemos dividido 200 imágenes para el entrenamiendo, y 50 para la validación.

Nota: Para movernos dentro de las carpetas de drive, podemos hacer uso del siguiente comando.

```shell
%cd drive/MyDrive/<folder>/<sub-folder>/<sub-sub-folder>/
```

### Entrenamiento

En cuanto al entrenamiendo, bastará con que ejecutemos el siguiente bloque de comandos. Es de destacar que sin un ordenador con soporte para _GPU_ y las librerías adecuadas instaladas, no podremos ejecutarlo. En nuestro caso realizamos el entrenamiendo con 70 etapas para no agotar los recursos de google colab.

```python
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="taxi")
trainer.setTrainConfig(object_names_array=["taxi"], batch_size=4, num_experiments=70, train_from_pretrained_model="pretrained-yolov3.h5")
trainer.trainModel()
```

Una vez finalizado el entrenamiento, obtenemos numerosas carpetas que el modelo ha creado en nuestro directorio de trabajo. En concreto, la carpeta `models` es donde se almacenan las iteraciones del modelo, y cogeremos la última pues esta será la que tiene menor tasa de error. 

### Evaluación 

Para evaluar el modelo, simplemente deberemos ejecutar el siguiente bloque de código, sustituyendo los nombres de los archivos a cargar y el nombre del archivo a guardar en la línea de `detections`.

```python
from imageai.Detection.Custom import CustomObjectDetection

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath("taxi/models/detection_model-ex-057--loss-0002.068.h5") 
detector.setJsonPath("taxi/json/detection_config.json")
detector.loadModel()

detections = detector.detectObjectsFromImage(input_image="taxi/taxi_goal.jpg", output_image_path="taxi-detected.jpg")

for detection in detections:
    print(detection["name"], " : ", detection["percentage_probability"], " : ", detection["box_points"])

```


En la siguiente tabla se muestran algunos resultados del modelo.

| Original        | Resultado           |
| :-------------: |:-------------:|
| ![alt_text](./deteccion/taxi_goal2.jpg)      | ![alt_text](./outputs/taxi-detected2.jpg) |
| ![alt_text](./deteccion/taxi_goal.jpg)      | ![alt_text](./outputs/taxi-detected.jpg) |

## Conclusiones

Únicamente partiendo de 5 imágenes iniciales, y generando a partir de ellas 200 imágenes de entrenamiento, hemos logrado entrenar una red neuronal capaz de detectar taxis. Si dispusiéramos de un mayor conjunto de entrenamiento, junto con una mayor capacidad computacional, aplicando las técnicas de _data augmentation_ que se presentan en este informe sin duda pueden conseguirse resultados realmente precisos.

## Recursos

Para la elaboración de este informe, nos hemos basado en los siguientes artículos:

* Para la creación del modelo: [Medium: Train Object Detection AI with 6 lines of code](https://medium.com/deepquestai/train-object-detection-ai-with-6-lines-of-code-6d087063f6ff)
* Para el etiquetado: [Medium: Object Detection Training — Preparing your custom dataset](https://medium.com/deepquestai/object-detection-training-preparing-your-custom-dataset-6248679f0d1d)

Todos los archivos necesarios, junto con el modelo generado, se pueden encontrar en el siguiente repositorio de Github.

[Github Object Recognition](https://github.com/Pablo-Dominguez/software-matematicas/tree/master/clases-python-v2/entrega)